In [34]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx
import seaborn as sns


import socnet as sn

from scipy import stats

import matplotlib.pyplot as plt

import math

import numpy as np

In [35]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

In [36]:
one = sn.load_graph('one-mode.gml')
two = sn.load_graph('two-mode.gml')
sn.show_graph(one, nlab=True)

../socnet.py:59: RuntimeWarning:

divide by zero encountered in double_scalars

../socnet.py:187: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:188: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:189: RuntimeWarning:

invalid value encountered in double_scalars

../socnet.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [37]:
sn.show_graph(two, nlab=True)

In [67]:
table = pd.DataFrame({
    'Aluno(a)': [one.nodes[n]['label'] for n in one.nodes],
    'In-Degree': [one.in_degree(n) for n in one.nodes],
    'Out-Degree': [one.out_degree(n) for n in one.nodes]
})

table.sort_values('In-Degree', ascending = False)

,Aluno(a),In-Degree,Out-Degree
10,Hugo,12,3
18,Graicer,10,2
7,Fred,7,4
8,Toyama,7,2
15,Forba,7,7
23,Chen,6,3
0,Rachel,5,4
9,Vitoria,5,4
22,Isa,3,2
5,Rapha,3,1


In [69]:
table = pd.DataFrame({
    'Gênero': [two.nodes[n]['label'] for n in two.nodes if two.out_degree(n) == 0],
    'In-Degree': [two.in_degree(n) for n in two.nodes if two.out_degree(n) == 0],
})

table.sort_values('In-Degree', ascending = False)

,Gênero,In-Degree
9,Eletronica,19
10,Pop Rock,11
3,Pop,11
14,Rock classico,11
2,Rap Nacional,10
13,Pagode,10
7,Funk,9
8,Sertanejo,8
6,Indie Rock,8
11,Disco,8


In [40]:
def norm_nodes(graph, table):
    '''
    Função que normaliza os nós de um grafo.
    
    Recebe como input um grafo e uma tabela com In-Degree ponderado e Out-Degree ponderado,
    e retorna: o novo grafo normalizado, lista com os valores dos In-Degree e Out-Degree
    '''
    insum = table['In-Degree ponderado'].sum()
    outsum = table['Out-Degree ponderado'].sum()
    normin = []
    normout = []
    for n in two.nodes:
        if graph.nodes[n]['Out-Degree ponderado'] == 0:
            graph.nodes[n]['In-Degree ponderado'] = two.nodes[n]['In-Degree ponderado']/insum
            normin.append(two.nodes[n]['In-Degree ponderado'])
        elif two.nodes[n]['In-Degree ponderado'] == 0:
            graph.nodes[n]['Out-Degree ponderado'] = two.nodes[n]['Out-Degree ponderado']/outsum
            normout.append(two.nodes[n]['Out-Degree ponderado'])
    return graph, normin+normout

In [41]:
# Calcular outdegree ponderado e indegree ponderado
count = 0
eucdiff = 1
while eucdiff > 0.000001:

    for n in two.nodes:
#         Somando todos os in degrees de cada gênero e todos out de cada pessoa
#         Calculando Indegree
        insum = 0
        outsum = 0
        if count == 0:
            for j in two.predecessors(n):
                insum += two.out_degree(j)
#             Definindo o indegree (de cada gênero) ponderado
            two.nodes[n]['In-Degree ponderado'] = insum

        else:
#             Recalculando e definindo o indegree (de cada gênero) ponderado
            for j in two.predecessors(n):
                insum += two.nodes[j]['Out-Degree ponderado']
            two.nodes[n]['In-Degree ponderado'] = insum

    for n in two.nodes:
#         Calculando Outdegree
        insum = 0
        outsum = 0
        if count == 0:
            for i in two.successors(n):
                outsum += two.in_degree(i)
#             Definindo o outdegree(de cada pessoa) ponderado
            two.nodes[n]['Out-Degree ponderado'] = outsum

        else:
#             Recalculando e definindo o outdegree (de cada pessoa) ponderado
            for i in two.successors(n):
                outsum += two.nodes[i]['In-Degree ponderado']
            two.nodes[n]['Out-Degree ponderado'] = outsum
            
#     Normalizando In/Outdegree
    table = pd.DataFrame({
        'In-Degree ponderado': [two.nodes[n]['In-Degree ponderado'] for n in two.nodes],
        'Out-Degree ponderado': [two.nodes[n]['Out-Degree ponderado'] for n in two.nodes]
    })
    two, oldlist = norm_nodes(two, table)

#     Recalculando In/Outdegree
    for n in two.nodes:
        insum = 0
        outsum = 0

#         Recalculando e definindo o indegree (de cada gênero) e o outdegree(de cada pessoa) ponderados
        for j in two.predecessors(n):
            insum += two.nodes[j]['Out-Degree ponderado']
        two.nodes[n]['In-Degree ponderado'] = insum

    for n in two.nodes:
        insum = 0
        outsum = 0

        for i in two.successors(n):
            outsum += two.nodes[i]['In-Degree ponderado']
        two.nodes[n]['Out-Degree ponderado'] = outsum

#     Renormalizando In/Outdegree 
    table = pd.DataFrame({
        'In-Degree ponderado': [two.nodes[n]['In-Degree ponderado'] for n in two.nodes],
        'Out-Degree ponderado': [two.nodes[n]['Out-Degree ponderado'] for n in two.nodes]
    })
    two, newlist = norm_nodes(two, table)
    

#     Diferença euclidiana
#     velhalista = normin + normout
#     novalista = normin2 + normout2
    eucsum = 0
    for k in range(len(oldlist)):
        eucsum += (oldlist[k] - newlist[k])**2
    eucdiff = math.sqrt(eucsum)
    
    count += 1

print("Iterações até estabilização:", count)


table = pd.DataFrame({
    'Gênero/Aluno': [two.nodes[n]['label'] for n in two.nodes],
    'In-Degree': [two.in_degree(n) for n in two.nodes],
    'In-Degree ponderado': [two.nodes[n]['In-Degree ponderado'] for n in two.nodes],
    'Out-Degree': [two.out_degree(n) for n in two.nodes],
    'Out-Degree ponderado': [two.nodes[n]['Out-Degree ponderado'] for n in two.nodes]
})

Iterações até estabilização: 5


In [43]:
alumni = pd.DataFrame({
    'Aluno': [two.nodes[n]['label'] for n in two.nodes  if two.in_degree(n) == 0],
    'Out-Degree': [two.out_degree(n) for n in two.nodes if two.in_degree(n) == 0],
    'Out-Degree ponderado': [two.nodes[n]['Out-Degree ponderado'] for n in two.nodes if two.in_degree(n) == 0],
})
alumni.sort_values('Out-Degree ponderado', ascending = False)

,Aluno,Out-Degree,Out-Degree ponderado
8,Forba,15,0.094578
19,Jean,10,0.069771
14,Sabrina,10,0.068451
13,Toyama,8,0.059988
12,Rachel,7,0.054039
10,Vitoria,7,0.053746
9,Hugo,7,0.051296
0,Isa,7,0.050524
22,Graicer,6,0.047240
18,Joao,6,0.046718


In [44]:
genres = pd.DataFrame({
    'Gênero': [two.nodes[n]['label'] for n in two.nodes  if two.out_degree(n) == 0],
    'In-Degree': [two.in_degree(n) for n in two.nodes if two.out_degree(n) == 0],
    'In-Degree ponderado': [two.nodes[n]['In-Degree ponderado'] for n in two.nodes if two.out_degree(n) == 0],
})
genres.sort_values('In-Degree ponderado', ascending = False)

,Gênero,In-Degree,In-Degree ponderado
9,Eletronica,19,0.127928
14,Rock classico,11,0.080279
10,Pop Rock,11,0.078177
3,Pop,11,0.074460
13,Pagode,10,0.072051
2,Rap Nacional,10,0.067788
6,Indie Rock,8,0.064503
11,Disco,8,0.064144
8,Sertanejo,8,0.057060
7,Funk,9,0.056638


In [45]:
indegree = nx.in_degree_centrality(two)

values = []
for key, value in indegree.items():
    if value != 0:
        values.append(value)

ind = []
for n in two.nodes:
    if two.out_degree(n) == 0:
        ind.append(two.in_degree(n))

In [46]:
indegree = nx.in_degree_centrality(two)
# indegree = sorted(indegree.items(), key=lambda x: x[1])

genres = pd.DataFrame({
    'Gênero': [two.nodes[n]['label'] for n in two.nodes  if two.out_degree(n) == 0],
    'In-Degree': [two.in_degree(n) for n in two.nodes if two.out_degree(n) == 0],
    'In-Degree ponderado': [value for key, value in indegree.items() if value != 0],
})
genres.sort_values('In-Degree ponderado', ascending = False)

,Gênero,In-Degree,In-Degree ponderado
9,Eletronica,19,0.487179
10,Pop Rock,11,0.282051
3,Pop,11,0.282051
14,Rock classico,11,0.282051
2,Rap Nacional,10,0.256410
13,Pagode,10,0.256410
7,Funk,9,0.230769
8,Sertanejo,8,0.205128
6,Indie Rock,8,0.205128
11,Disco,8,0.205128


In [59]:
# Calculando  closeness e betweenness
bc = nx.betweenness_centrality(one)
cc = nx.closeness_centrality(two)

In [58]:
table = pd.DataFrame({
    'Gênero/Aluno': [two.nodes[n]['label'] for n in two.nodes],
    'In-Degree': [two.in_degree(n) for n in two.nodes],
    'In-Degree ponderado': [two.nodes[n]['In-Degree ponderado'] for n in two.nodes],
    'Out-Degree': [two.out_degree(n) for n in two.nodes],
    'Out-Degree ponderado': [two.nodes[n]['Out-Degree ponderado'] for n in two.nodes],
    'Closeness' : [cc[n] for n in two.nodes]
})

table.sort_values('Closeness', ascending=False)

,Gênero/Aluno,In-Degree,In-Degree ponderado,Out-Degree,Out-Degree ponderado,Closeness
24,Eletronica,19,0.127928,0,0.000000,0.487179
25,Pop Rock,11,0.078177,0,0.000000,0.282051
5,Pop,11,0.074460,0,0.000000,0.282051
32,Rock classico,11,0.080279,0,0.000000,0.282051
31,Pagode,10,0.072051,0,0.000000,0.256410
4,Rap Nacional,10,0.067788,0,0.000000,0.256410
19,Funk,9,0.056638,0,0.000000,0.230769
22,Sertanejo,8,0.057060,0,0.000000,0.205128
26,Disco,8,0.064144,0,0.000000,0.205128
17,Indie Rock,8,0.064503,0,0.000000,0.205128


In [64]:
alumni = pd.DataFrame({
    'Aluno': [one.nodes[n]['label'] for n in one.nodes],
     'Betweenness' : [bc[n] for n in one.nodes],
})
alumni.sort_values('Betweenness', ascending = False)

,Aluno,Betweenness
9,Vitoria,0.204106
23,Chen,0.200272
7,Fred,0.197162
15,Forba,0.193961
14,Sabrina,0.189855
2,Bruna,0.157035
10,Hugo,0.087258
18,Graicer,0.066486
0,Rachel,0.059541
22,Isa,0.052355
